### 可选问题-预测北京房价
#### 北京房价数据集： bj_housing.csv
免责声明：考虑到北京房价受到宏观经济、政策调整等众多因素的直接影响，预测结果仅供参考。

这个数据集的特征有：

    Area：房屋面积，平方米   
    Room：房间数，间  
    Living: 厅数，间  
    School: 是否为学区房，0或1   
    Year: 房屋建造时间，年  
    Floor: 房屋所处楼层，层     
目标变量：    
    Value: 房屋人民币售价，万   
     
你可以参考上面学到的内容，拿这个数据集来练习    

        数据分割与重排、   
        定义衡量标准、  
        训练模型、  
        评价模型表现、   
        使用网格搜索配合交叉验证对参数进行调优并选出最佳参数，   
        比较两者的差别，最终得出最佳模型对验证集的预测分数。  



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [2]:
# 载入北京房屋的数据集
data = pd.read_csv('bj_housing.csv')
prices = data['Value']
features = data.drop('Value', axis=1)

print('Beijing housing dataset has {} data points with {} variables each'.format(*features.shape))

Beijing housing dataset has 9999 data points with 6 variables each


In [3]:
# 分割训练集、测试集
X_train, X_test, y_train, y_test = train_test_split(features, prices, test_size=0.2, random_state = 30)
print('X_train Size:', X_train.shape[0])
print('X_test Size:', X_test.shape[0])

X_train Size: 7999
X_test Size: 2000


In [4]:
# 定义衡量标准：决定系数 Coefficient of Determination
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

In [5]:
# 训练模型
def fit_model(X, y):
    cross_validator = KFold(n_splits=10, random_state=1, shuffle=False)
    regressor = DecisionTreeRegressor()
    params = {'max_depth': range(1,11)}
    scoring_fnc = make_scorer(performance_metric)
    grid = GridSearchCV(regressor, params, scoring=scoring_fnc, cv=cross_validator)
    
    # 基于输入数据[X, y]，进行网格搜索
    grid = grid.fit(X, y)
    
    # 返回网格搜索后的最优模型
    return grid.best_estimator_

In [9]:
# 基于训练数据，获取最优模型
optimal_reg = fit_model(X_train, y_train)

# 输出最优模型的'max_depth'参数
print("Parameter 'max_depth' is {} for the optimal model.".format(optimal_reg.get_params()['max_depth']))

Parameter 'max_depth' is 5 for the optimal model.


这里返回的最佳参数的值，会随着 random_state 在 train_test_split, Kfold （如果设置shuffle=True） 和 DecisionTreeRegressor 中的取值变化而变化。
要得到一致结果，每个随机数都需要设定

In [10]:
# 计算相对于目标变量的决定系数的R2的值
predicted_price = optimal_reg.predict(X_test)

r2 = performance_metric(y_test, predicted_price)
print('Optimal model has R^2 score {:,.2f} on test data'.format(r2))

Optimal model has R^2 score 0.75 on test data
